In [2]:
import os
import re
import time
import datetime
import requests
import pandas as pd
import numpy as np
from datetime import datetime
from bs4 import BeautifulSoup, Comment
from selenium import webdriver
from IPython.display import clear_output
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains

In [2]:
def foreigner(html):
    soup = BeautifulSoup(html,'html.parser',from_encoding='utf-8')
    links = soup.find_all('p')
    # print(links)
    # <span itemprop="birthPlace">

    links = soup.find_all(['span'] , attrs = { 'itemprop' : 'birthPlace'})
    for link in links:
        for c in (link.descendants):
            aa = ((c.find_next('a')).attrs)
            if 'country=US' in aa['href']:
                return False
    return True

## Should exccute the chromedriver and get the page source

In [3]:
def league(html):
    result = []
    soup = BeautifulSoup(html,'html.parser',from_encoding='utf-8')
    links = soup.find_all('div', attrs = {'class': 'leaderboard_wrapper' , 'id': 'all_leaderboard'})

    comments = soup.findAll(text=lambda text:isinstance(text, Comment))
    for comment in comments:
        a = comment.find('div_leaderboard')
        if (a != -1):
            #print(a,comment)
            break
            
    ts = BeautifulSoup(comment.string, 'html.parser')
    ts = ts.find_all('div', {'id' : 'leaderboard_all_league' , 'class' : 'data_grid_box'})
    for tt in ts:
        kk = tt.table.find_all('a')
        for k in kk:
            result.append((k.text))
    return result

## Should exceute the chromedriver and get the page source

In [4]:
def allstar(html):
    result = []
    soup = BeautifulSoup(html,'html.parser',from_encoding='utf-8')
    links = soup.find_all('div', attrs = {'class': 'leaderboard_wrapper' , 'id': 'all_leaderboard'})

    comments = soup.findAll(text=lambda text:isinstance(text, Comment))
    for comment in comments:
        a = comment.find('div_leaderboard')
        if (a != -1):
            #print(a,comment)
            break
    ts = BeautifulSoup(comment.string, 'html.parser')
    ts = ts.find_all('div', {'id' : 'leaderboard_allstar' , 'class' : 'data_grid_box'})
    for tt in ts:
        kk = tt.table.find_all('a')
        for k in kk:
            result.append((k.text))
            
    result = list(map (lambda x : str(int(x.split()[0])-1 ) + '-' + (x.split()[0][2:])  , result))
    return result

In [5]:
def Hall(html):
    soup = BeautifulSoup(html,'html.parser',from_encoding='utf-8')
    links = soup.find_all('li' , {'class'  : 'bling_special bling_hof'})
    for link in links:
        if link.text == "Hall of Fame":
            return True

In [6]:
def btn_click(wd,url):
    wd.get(url)
    script = "vjs_addClass(document.getElementById('leaderboard_allstar'),'show_all')"
    script2 = "vjs_addClass(document.getElementById('leaderboard_allleague'),'show_all')"
    btn = wd.execute_script(script)
    btn = wd.execute_script(script2)
    return(wd.page_source)

In [7]:
def get_all_info(wd,url):
    html = btn_click(wd,url)
    result = ([Hall(html) ,league(html), allstar(html), foreigner(html)])
    return (result)

In [8]:
all_data = []
df = pd.read_csv('player_my_df.csv')
info = pd.read_excel('info.xlsx')

dirs = !ls ./data
dirs_no = sorted(list(map (lambda x : int(x.split('.')[0]),dirs)))

In [117]:
clear_output()
pass

# chrome_options.add_argument("--headless") # make the chrome invisible
chrome_options = Options() 
chrome_options.add_argument("--window-size=100x10")
wd = webdriver.Chrome(options=chrome_options)

# empty = pd.Series(['**' ,'**',['**'],['**']])
empty = (['**' ,'**',['**'],['**']])
df_remained = pd.DataFrame()
none = pd.DataFrame([None,[],[],False])

df = pd.DataFrame(df)
row, col = df.shape

all_url = (df['5'])
for i in range(4044, len(all_url)):
    print(i,dirs_no[0],len(all_data))
    if len(dirs_no) >= 0:
        if dirs_no[0] != i:
            #df_remained.append(empty,ignore_index=True)
            all_data.append(empty)
            continue
        else:
            dirs_no.pop(0)
        
    url = all_url[i]
    appended_data = get_all_info(wd,url)
    all_data.append(appended_data)
    
    if i % 100 == 0:
        print(i)
        clear_output()
        df_remained.to_excel('remain.xlsx') 

KeyError: '5'

In [ ]:
chrome_options = Options() 
chrome_options.add_argument("--window-size=100x10")
wd = webdriver.Chrome(options=chrome_options)

sample_url = "https://www.basketball-reference.com/players/g/greenac01.html"
get_all_info(wd,sample_url)

## Load the all data processed

In [5]:
import pickle
dirs = !ls ./data
dirs_no = sorted(list(map (lambda x : int(x.split('.')[0]),dirs)))

with open ('all_data', 'rb') as fp:
    data = pickle.load(fp)
      
info = pd.read_excel('info.xlsx')

In [12]:
df = pd.DataFrame()   
row = len(data)

for i in range(row):
    if(len(dirs_no) > 0):
        if i != dirs_no:
            continue
        dirs_no[0].pop()

    print(i)
    my_list = pd.read_html('data/' + str(i) + '.xls')
    row , col = ((my_list[0].shape))
    small_df = pd.DataFrame(np.array(my_list[0]).reshape(row , col))
    small_df = small_df.dropna(axis=1)

    ## Process fix award ##
    ## [None, ['1988-89'], ['1989-90'], False]
    award = data[i]
    inf = list(info.iloc[i,:])
    ind = info.columns

    if(award[0]):
        small_df['名人堂']=(pd.Series([1]*row))
    else:
        small_df['名人堂']= (pd.Series([0]*row))

    if(award[3]):
        small_df['外籍球員']=(pd.Series([1]*row))
    else:
        small_df['外籍球員']= (pd.Series([0]*row))

    ## Process award ##
    print(hi)

    ## Process info ##
    for k in range(len(inf)):
        print(inf[k])
        small_df[ind[k]]= (inf[k])
    print(small_df)
#         break

    print(info.iloc[i,:].index)
    print(small_df)
    print(data[i])
#         break

(     FirstName    LastName  From    To        位置  呎   吋    磅  \
 0         A.C.       Green  1986  2001     PF-SF  6   9  220   
 1         A.C.       Green  2000     -         C  6  10  227   
 2         A.J.     English  1991  1992        SG  6   3  175   
 3         A.J.      Guyton  2001  2003        PG  6   1  180   
 4         A.J.      Guyton  2017     -         C  7   0  260   
 5         A.J.    Matthews   NaN   NaN       F-C  7   1  220   
 6         A.J.       Price  2010  2015     SG-PG  6   2  181   
 7         A.J.   Slaughter   NaN   NaN         G  6   3  185   
 8         A.J.   Slaughter  1991     -        PG  6   2  180   
 9         A.J.   Slaughter  1971     -        SF  6   7  210   
 10       Aaron      Brooks  2008  2018        PG  6   0  161   
 11       Aaron       Craft   NaN   NaN         G  6   2  195   
 12       Aaron      Gordon  2015  2019     PF-SF  6   9  220   
 13       Aaron        Gray  2008  2014         C  7   0  270   
 14       Aaron    Harris